In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns
import datetime as dt

In [2]:
final_invoice = pd.read_pickle('pickled/final_invoice.pkl')

In [3]:
final_invoice.columns

Index(['Unnamed: 0', 'Amt Rcvd From Custom', 'Amt Rcvd From Ins Co',
       'Area / Locality', 'CGST(14%)', 'CGST(2.5%)', 'CGST(6%)', 'CGST(9%)',
       'CITY', 'Cash /Cashless Type', 'Claim No.', 'Cust Type', 'Customer No.',
       'District', 'Expiry Date', 'Gate Pass Date', 'Gate Pass Time',
       'IGST(12%)', 'IGST(18%)', 'IGST(28%)', 'IGST(5%)', 'Insurance Company',
       'Invoice Date', 'Invoice No', 'Invoice Time', 'Job Card No',
       'JobCard Date', 'JobCard Time', 'KMs Reading', 'Labour Total', 'Make',
       'Misc Total', 'Model', 'ODN No.', 'OSL Total', 'Order Type',
       'Outstanding Amt', 'Parts Total', 'Pin code', 'Plant', 'Plant Name1',
       'Policy no.', 'Print Status', 'Recovrbl Exp', 'Regn No',
       'SGST/UGST(14%)', 'SGST/UGST(2.5%)', 'SGST/UGST(6%)', 'SGST/UGST(9%)',
       'Service Advisor Name', 'TDS amount', 'Technician Name',
       'Total Amt Wtd Tax.', 'Total CGST', 'Total GST', 'Total IGST',
       'Total SGST/UGST', 'Total Value', 'User ID'],
     

In [4]:
rfm0 = final_invoice[['Customer No.', 'Invoice Date', 'Invoice Time', 'Invoice No', 'Total Amt Wtd Tax.']]

In [5]:
rfm0['date'] = pd.to_datetime(rfm0['Invoice Date'] + ' ' + rfm0['Invoice Time'])

rfm0.drop(['Invoice Date', 'Invoice Time'], axis='columns',inplace=True)

<ipython-input-5-225e909af387>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm0['date'] = pd.to_datetime(rfm0['Invoice Date'] + ' ' + rfm0['Invoice Time'])
C:\Users\Baby\Anaconda3\envs\py38-basic-ml\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [6]:
rfm0.drop_duplicates()

PRESENT = dt.datetime(2016,11,1)

rfm = rfm0.groupby('Customer No.').agg({'date': lambda date: (PRESENT - date.max()).days,
                                       'Invoice No': 'count',
                                       'Total Amt Wtd Tax.': lambda rev: rev.sum(),})
rfm.columns = ['recency', 'frequency', "monetary"]
rfm['recency'] = rfm['recency'].astype(int)

rfm['r_quartile'] = pd.qcut(rfm['recency'], 4, ['1', '2', '3', '4'])  # 1 Best
rfm['f_quartile'] = pd.cut(rfm['frequency'], 4, labels=['4', '3', '2', '1'])  # 1 Best
rfm['m_quartile'] = pd.qcut(rfm['monetary'], 4, ['4', '3', '2', '1'])  # 1 Best

rfm['RFM_score'] = rfm.r_quartile.astype(str)+ rfm.f_quartile.astype(str) + rfm.m_quartile.astype(str)

In [7]:
rfm.columns = ['num_days','num_transactions','spent_money', 'r_quartile', 'f_quartile', 'm_quartile', 'RFM_score']

In [8]:
rfm.head()

,num_days,num_transactions,spent_money,r_quartile,f_quartile,m_quartile,RFM_score
Customer No.,,,,,,,
1,1311,4,0.00,4,4,4,444
10,1232,5,11105.56,4,4,1,441
100,1510,2,20799.74,4,4,1,441
1000,748,6,8813.44,4,4,1,441
10000,436,7,1735.52,3,4,3,343


In [9]:
rfm.shape

(253484, 7)

In [10]:
rfm['avg_order_value']=rfm['spent_money']/rfm['num_transactions']

In [11]:
rfm.head()

,num_days,num_transactions,spent_money,r_quartile,f_quartile,m_quartile,RFM_score,avg_order_value
Customer No.,,,,,,,,
1,1311,4,0.00,4,4,4,444,0.000000
10,1232,5,11105.56,4,4,1,441,2221.112000
100,1510,2,20799.74,4,4,1,441,10399.870000
1000,748,6,8813.44,4,4,1,441,1468.906667
10000,436,7,1735.52,3,4,3,343,247.931429


In [26]:
"""
Customer Value = Average Order Value * Purchase Frequency

Average Order Value = Total Revenue / Total Number of Orders

Purchase Frequency =  Total Number of Orders / Total Number of Customers

Customer Lifetime=1/Churn Rate

Churn Rate= 1-Repeat Rate
"""

'\nCustomer Value = Average Order Value * Purchase Frequency\n\nAverage Order Value = Total Revenue / Total Number of Orders\n\nPurchase Frequency =  Total Number of Orders / Total Number of Customers\n\nCustomer Lifetime=1/Churn Rate\n\nChurn Rate= 1-Repeat Rate\n'

In [12]:
purchase_frequency=sum(rfm['num_transactions'])/rfm.shape[0]
purchase_frequency

1.9421896451058054

In [13]:
repeat_rate=rfm[rfm.num_transactions > 1].shape[0]/rfm.shape[0]
repeat_rate

0.37535702450647773

In [14]:
churn_rate=1-repeat_rate
churn_rate

0.6246429754935223

In [15]:
# CLTV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin.

# Customer Value = Average Order Value * Purchase Frequency

In [16]:
rfm['profit_margin']=rfm['spent_money']*0.05

In [17]:
# Customer Value
rfm['CLV']=(rfm['avg_order_value']*purchase_frequency)/churn_rate

In [18]:
#Customer Lifetime Value
rfm['cust_lifetime_value']=rfm['CLV']*rfm['profit_margin']

In [19]:
rfm.head()

,num_days,num_transactions,spent_money,r_quartile,f_quartile,m_quartile,RFM_score,avg_order_value,profit_margin,CLV,cust_lifetime_value
Customer No.,,,,,,,,,,,
1,1311,4,0.00,4,4,4,444,0.000000,0.000,0.000000,0.000000e+00
10,1232,5,11105.56,4,4,1,441,2221.112000,555.278,6906.058174,3.834782e+06
100,1510,2,20799.74,4,4,1,441,10399.870000,1039.987,32336.103369,3.362913e+07
1000,748,6,8813.44,4,4,1,441,1468.906667,440.672,4567.241496,2.012655e+06
10000,436,7,1735.52,3,4,3,343,247.931429,86.776,770.888127,6.689459e+04


In [20]:
rfm['cust_lifetime_value']

Customer No.
1         0.000000e+00
10        3.834782e+06
100       3.362913e+07
1000      2.012655e+06
10000     6.689459e+04
              ...     
PLX241    1.879886e+06
PLX277    7.339447e+06
PLX312    5.012110e+04
PLX380    2.937684e+06
PLX386    0.000000e+00
Name: cust_lifetime_value, Length: 253484, dtype: float64

In [21]:
#rfm['LTV_cluster'] = pd.qcut(rfm['cust_lifetime_value'], 2, ['0', '1'])  # 1 Best

In [22]:
#rfm.head()

In [23]:
#from sklearn.cluster import KMeans

#kmeans = KMeans(n_clusters=2)
#kmeans.fit(rfm[['cust_lifetime_value']])
#rfm['cluster'] = kmeans.predict(rfm[['cust_lifetime_value']])

In [24]:
#rfm.head()

In [25]:
#rfm.groupby('cluster')['cust_lifetime_value'].describe()